# Lasso Method Features Selection 

Preparation

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import mutual_info_classif

PROJECT_ROOT = Path("..").resolve()
IN_CSV = PROJECT_ROOT / "outputs" / "extracted_features.csv"

OUT_DIR = PROJECT_ROOT / "outputs"
RANK_DIR = OUT_DIR / "rankings"
DATASET_DIR = OUT_DIR / "datasets"
RANK_DIR.mkdir(parents=True, exist_ok=True)
DATASET_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(IN_CSV)

feature_cols = ["CNT","CRL","SKEN","KTS","VAR","STD","ENT","EG","MN","HGN","RMS","SM","IDM"]
X = df[feature_cols].values
y = df["Output"].astype(int).values

print("Loaded:", IN_CSV)
print("X:", X.shape, "| y:", y.shape)


Loaded: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\extracted_features.csv
X: (724, 13) | y: (724,)


Scaling (For Lasso) pakai Min max Scaling

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print("Scaled X:", X_scaled.shape, "| min:", X_scaled.min(), "| max:", X_scaled.max())


Scaled X: (724, 13) | min: 0.0 | max: 1.0000000000000018


Fit L1 Logistic Regression (LASSO)

In [ ]:

alphas = np.logspace(-4, -1, 30)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=20000)
lasso_cv.fit(X_scaled, y)
best_alpha = float(lasso_cv.alpha_)
print("Best alpha (CV):", best_alpha)

lasso = Lasso(alpha=best_alpha, random_state=42, max_iter=20000)
lasso.fit(X_scaled, y)

coef = np.abs(lasso.coef_)
lasso_rank = (
    pd.DataFrame({"feature": feature_cols, "score": coef})
      .sort_values("score", ascending=False)
      .reset_index(drop=True)
)


lasso_rank_path = RANK_DIR / "lasso_rank.csv"
lasso_rank.to_csv(lasso_rank_path, index=False)
print("Saved:", lasso_rank_path)

)
fig, ax = plt.subplots(figsize=(7, 3.5))
ax.bar(lasso_rank["feature"], lasso_rank["score"])
ax.set_title("LASSO Rank (abs coef)")
ax.set_xlabel("Feature")
ax.set_ylabel("Score")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

lasso_rank


Best alpha (CV): 0.0001
Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\rankings\lasso_rank.csv


feature     score
0      CNT  1.352424
1      STD  0.876128
2      KTS  0.688741
3       SM  0.566730
4     SKEN  0.547445
5      ENT  0.492333
6      VAR  0.402897
7      RMS  0.232559
8       EG  0.225123
9      CRL  0.219327
10     HGN  0.161046
11      MN  0.000000
12     IDM  0.000000

In [ ]:

mi = mutual_info_classif(X_scaled, y, random_state=42)

mi_rank = (
    pd.DataFrame({"feature": feature_cols, "score": mi})
      .sort_values("score", ascending=False)
      .reset_index(drop=True)
)

mi_rank_path = RANK_DIR / "mutual_info_rank.csv"
mi_rank.to_csv(mi_rank_path, index=False)
print("Saved:", mi_rank_path)

fig, ax = plt.subplots(figsize=(7, 3.5))
ax.bar(mi_rank["feature"], mi_rank["score"])
ax.set_title("Mutual Information Rank")
ax.set_xlabel("Feature")
ax.set_ylabel("MI score")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

mi_rank


Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\rankings\mutual_info_rank.csv


feature     score
0       MN  0.319487
1       SM  0.311769
2      STD  0.311572
3      VAR  0.311572
4      RMS  0.302394
5      KTS  0.290726
6     SKEN  0.273838
7      ENT  0.214745
8      CRL  0.170082
9      CNT  0.095635
10     HGN  0.073659
11     IDM  0.073659
12      EG  0.050344

Top-K CSV LASSO (Top 5/7/9/10)

In [ ]:
TOP_K_LIST = [5, 7, 9, 10]


meta_cols = [c for c in ["ClassName", "orig_path", "prep_path"] if c in df.columns]

for k in TOP_K_LIST:
    top_feats = lasso_rank["feature"].head(k).tolist()
    out_cols = top_feats + ["Output"] + meta_cols
    out_df = df[out_cols].copy()
    out_path = DATASET_DIR / f"data{k}L.csv"   # L = LASSO
    out_df.to_csv(out_path, index=False)
    print("Saved:", out_path, "| cols:", len(out_df.columns))


Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\datasets\data5L.csv | cols: 9
Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\datasets\data7L.csv | cols: 11
Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\datasets\data9L.csv | cols: 13
Saved: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\datasets\data10L.csv | cols: 14
